# OR Advent 2024 December 4: meeting rooms (Kotlin)

## Assignment

Hi, I'm SalvadOR, responsible for creating a school timetable to organize classes, teachers, and rooms for an upcoming semester.

We have some strict requirements to meet. This is a very hard problem we face every year, and I need your help to design an optimal timetable.

Here's the situation:

There are 4 classes, each requiring specific teaching sessions.
There are 4 teachers, and each teacher has assigned subjects to teach.
There are 4 rooms, and only one class can occupy a room during any given period.
The timetable spans 30 periods, and we must ensure that all requirements are met without any conflicts or overlaps.
I desperately need a timetable that satisfies all requirements (each class meets with the right teacher in the right room the required number of times), avoiding any type of clashes such as double-booking a teacher, room, or class during the same period.

We say a timetable is optimized when it minimizes idle periods and maximizes resource utilization (teachers and rooms).

Can you help me solve this problem?

## Note

There is very little to optimize here, since there are no idle periods nor teachers/room resource utilization problems.
**This was mostly a test for me (Tom) to see how Kotlin notebooks work.**

In [110]:
val roomCount = 4


## Dependencies

This Jupyter Notebook solves an OR Advent problem with [Timefold Solver](https://timefold.ai/open-source-solver), the open source planning solver AI. We add it as a dependency:

In [111]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.16.0")


## Domain

We need to assign lessons to timeslots:

In [112]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.variable.PlanningVariable


data class Timeslot(val id : Int, val dayIndex: Int, val slotIndex: Int) {
    // No-arg constructor required for Timefold
    constructor() : this(0,0,0)
}

In [113]:
import java.time.LocalDateTime

@PlanningEntity
data class Lesson (
    @PlanningId val id: Int,
    val teacher: String,
    val room: String,
    val classGroup: String) {

    @PlanningVariable
    var timeslot: Timeslot? = null
    constructor() : this(0,"","","")
    
}

## Constraints

The solver takes into account hard and soft constraints:

In [114]:
import ai.timefold.solver.core.api.score.buildin.hardmediumsoftlong.HardMediumSoftLongScore
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore
import ai.timefold.solver.core.api.score.stream.*
import ai.timefold.solver.core.api.score.stream.bi.BiConstraintStream
import java.time.Duration

class OrAdventConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint>? {
        return arrayOf(
            // Hard constraints
            roomConflict(constraintFactory),
            teacherConflict(constraintFactory),
            classGroupConflict(constraintFactory),
        )
    }

    fun roomConflict(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEachUniquePair(
            Lesson::class.java,
            // ... in the same timeslot ...
            Joiners.equal(Lesson::timeslot),
            // ... in the same room ...
            Joiners.equal(Lesson::room)
        )
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Room conflict");
    }

    fun teacherConflict(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEachUniquePair(
            Lesson::class.java,
            // ... in the same timeslot ...
            Joiners.equal(Lesson::timeslot),
            // ... with the same teacher ...
            Joiners.equal(Lesson::teacher)
        )
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Teacher conflict");
    }

    fun classGroupConflict(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.forEachUniquePair(
            Lesson::class.java,
            // ... in the same timeslot ...
            Joiners.equal(Lesson::timeslot),
            // ... with the same class group ...
            Joiners.equal(Lesson::classGroup)
        )
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Class group conflict");
    }
}

### The dataset class

This class represents the dataset to solve:

In [115]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

@PlanningSolution
data class OrAdventDataset(
    
    @ValueRangeProvider
    val timeslots: List<Timeslot>,

    @PlanningEntityCollectionProperty
    val lessons: List<Lesson>) {

    @PlanningScore
    var score: HardSoftScore? = null

    // No-arg constructor required for Timefold
    constructor() : this(emptyList(), emptyList())
}

## Input reader

In [116]:
import java.io.File

fun readDataset(): OrAdventDataset {
    val timeslots = mutableListOf<Timeslot>()

    val input: String = File("./instance_list.txt").readText()
    val lines = input.lines().filter {
        it.isNotBlank() && !it.startsWith("#") // Skip empty or commented lines
    }
    val firstLine : String = lines.get(0);
    val digits = Regex("\\d+").findAll(firstLine).map { it.value.toInt() }.toList()
    val (periodsPerDay, daysPerWeek) = digits

    var id = 0;
    for (period in 1..periodsPerDay) {
        for (day in 1..daysPerWeek) {
            timeslots.add(Timeslot(id, day, period))
            id++;
        }
    }

    id = 0;
    val lessons = mutableListOf<Lesson>()
    val regex = Regex("Schedule (\\w+) taught (\\w+) by (\\w+) in (\\w+) (\\d+) times")
    lines.drop(1).forEach { line ->
        val result = regex.find(line);
        if(result != null) {
            val (classId, subjectId, teacherId, roomId, times) = result.destructured
            lessons.add(Lesson(id, teacherId, roomId, classId))
            id++;
        }
    }
    return OrAdventDataset(timeslots, lessons)
}

## Solve it

Configure and run the solver:

In [117]:
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver
import ai.timefold.solver.core.config.solver.termination.TerminationConfig

val solverFactory: SolverFactory<OrAdventDataset> = SolverFactory.create(SolverConfig()
        .withSolutionClass(OrAdventDataset::class.java)
        .withEntityClasses(Lesson::class.java)
        .withConstraintProviderClass(OrAdventConstraintProvider::class.java)
        // Stop the solver if no better solution is found for 3 seconds.
        .withTerminationConfig(TerminationConfig().withUnimprovedSecondsSpentLimit(3L)))

val problem: OrAdventDataset = readDataset()

println("Solving the problem ...")
val solver: Solver<OrAdventDataset> = solverFactory.buildSolver()
val solution: OrAdventDataset = solver.solve(problem)
println("Solving finished with score (${solution.score}).")

Solving the problem ...
Solving finished with score (0hard/0soft).


## Visualize the solution

In [119]:
val lessonsOnTimeslot = solution.lessons.groupBy { t -> t.timeslot?.id }
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${solution.score}</p>")
    append("<p><b>By room</b></p>")
    append("<table><tr><th/>")
    for (i in 1 until roomCount+1) {
        append("<th>${i}</th>")
    }
    append("</tr>")
    for (timeslotKey in lessonsOnTimeslot.keys.sortedBy { i -> i }) {
        append("<tr><th>${timeslotKey}</th>")
        for (lesson in lessonsOnTimeslot.get(timeslotKey)!!.sortedBy { timeslot -> timeslot.room }) {
            append("<td>")
            append("Teacher:" + lesson.teacher + "<br/>")
            append("Class:" + lesson.classGroup)
            append("</td>")
        }
        append("</tr>")
    }
    append("</table>")
})

Score: 0hard/0soft By room 1 2 3 4 0 Teacher:T4 Class:C3 Teacher:T1 Class:C1 Teacher:T3 Class:C4 Teacher:T2 Class:C2 1 Teacher:T2 Class:C4 Teacher:T4 Class:C2 Teacher:T1 Class:C1 Teacher:T3 Class:C3 2 Teacher:T1 Class:C1 Teacher:T2 Class:C2 Teacher:T3 Class:C3 Teacher:T4 Class:C4 3 Teacher:T4 Class:C2 Teacher:T1 Class:C3 Teacher:T2 Class:C4 Teacher:T3 Class:C1 4 Teacher:T4 Class:C1 Teacher:T3 Class:C2 Teacher:T1 Class:C4 Teacher:T2 Class:C3 5 Teacher:T2 Class:C3 Teacher:T1 Class:C4 Teacher:T3 Class:C2 Teacher:T4 Class:C1 6 Teacher:T4 Class:C3 Teacher:T3 Class:C1 Teacher:T1 Class:C4 Teacher:T2 Class:C2 7 Teacher:T4 Class:C4 Teacher:T1 Class:C1 Teacher:T3 Class:C2 Teacher:T2 Class:C3 8 Teacher:T3 Class:C4 Teacher:T2 Class:C1 Teacher:T1 Class:C3 Teacher:T4 Class:C2 9 Teacher:T2 Class:C1 Teacher:T3 Class:C2 Teacher:T4 Class:C4 Teacher:T1 Class:C3 10 Teacher:T4 Class:C3 Teacher:T3 Class:C2 Teacher:T1 Class:C4 Teacher:T2 Class:C1 11 Teacher:T3 Class:C4 Teacher:T2 Class:C2 Teacher:T4 Class:C3 Teacher:T1 Class:C1 12 Teacher:T4 Class:C3 Teacher:T3 Class:C4 Teacher:T1 Class:C1 Teacher:T2 Class:C2 13 Teacher:T4 Class:C4 Teacher:T2 Class:C2 Teacher:T3 Class:C3 Teacher:T1 Class:C1 14 Teacher:T4 Class:C1 Teacher:T1 Class:C4 Teacher:T3 Class:C3 Teacher:T2 Class:C2 15 Teacher:T1 Class:C4 Teacher:T2 Class:C1 Teacher:T3 Class:C2 Teacher:T4 Class:C3 16 Teacher:T1 Class:C3 Teacher:T2 Class:C4 Teacher:T4 Class:C2 Teacher:T3 Class:C1 17 Teacher:T3 Class:C3 Teacher:T2 Class:C1 Teacher:T1 Class:C4 Teacher:T4 Class:C2 18 Teacher:T1 Class:C4 Teacher:T3 Class:C3 Teacher:T2 Class:C1 Teacher:T4 Class:C2 19 Teacher:T3 Class:C2 Teacher:T2 Class:C3 Teacher:T4 Class:C1 Teacher:T1 Class:C4 20 Teacher:T2 Class:C2 Teacher:T4 Class:C4 Teacher:T3 Class:C3 Teacher:T1 Class:C1 21 Teacher:T2 Class:C1 Teacher:T4 Class:C4 Teacher:T1 Class:C3 Teacher:T3 Class:C2 22 Teacher:T4 Class:C3 Teacher:T2 Class:C4 Teacher:T3 Class:C1 Teacher:T1 Class:C2 23 Teacher:T2 Class:C4 Teacher:T4 Class:C1 Teacher:T3 Class:C2 Teacher:T1 Class:C3 24 Teacher:T1 Class:C1 Teacher:T4 Class:C2 Teacher:T3 Class:C4 Teacher:T2 Class:C3 25 Teacher:T1 Class:C2 Teacher:T4 Class:C1 Teacher:T2 Class:C3 Teacher:T3 Class:C4 26 Teacher:T3 Class:C1 Teacher:T2 Class:C3 Teacher:T1 Class:C4 Teacher:T4 Class:C2 27 Teacher:T3 Class:C4 Teacher:T2 Class:C2 Teacher:T4 Class:C1 Teacher:T1 Class:C3 28 Teacher:T4 Class:C2 Teacher:T2 Class:C1 Teacher:T1 Class:C4 Teacher:T3 Class:C3 29 Teacher:T4 Class:C3 Teacher:T2 Class:C1 Teacher:T3 Class:C2 Teacher:T1 Class:C4

## Statistics

For a big dataset, a schedule visualization is often too verbose.
Let's visualize the solution through statistics:

In [120]:
%use kandy

In [121]:
val timeslotIds = solution.timeslots.map { it.id }
val lessonsInTimeslot = solution.timeslots.map { t -> solution.lessons.filter{ it.timeslot == t }.count() }

plot {
    layout.title = "Events per room"
    bars {
        x(timeslotIds) { axis.name = "Timeslots" }
        y(lessonsInTimeslot) { axis.name = "Lessons" }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="gD3uta"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Events per room"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"Timeslots",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Lessons",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0],
"y":[4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("gD3uta");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 1.5 
 
 
 
 
 
 
 2.0 
 
 
 
 
 
 
 2.5 
 
 
 
 
 
 
 3.0 
 
 
 
 
 
 
 3.5 
 
 
 
 
 
 
 4.0 
 
 
 
 
 
 
 
 
 Events per room 
 
 
 
 
 Lessons 
 
 
 
 
 Timeslots

This diagram is a bit silly ofcourse, since all lessons are always assigned.

## Analyze the score

Let's break down the score per constraint:

In [122]:
import ai.timefold.solver.core.api.solver.SolutionManager

val solutionManager = SolutionManager.create(solverFactory)
val scoreAnalysis = solutionManager.analyze(solution)

And visualize it:

In [123]:
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${scoreAnalysis.score}</p>")
    append("<ul>")
    for (constraint in scoreAnalysis.constraintMap().values) {
        append("<li>${constraint.constraintRef().constraintName}: ${constraint.score.toShortString()}</li>")
    }
    append("</ul>")
})

Score: 0hard/0soft Class group conflict: 0 Room conflict: 0 Teacher conflict: 0

## Notice

This code isn't optimized for benchmarking or scaling.

To learn more about planning optimization, visit [docs.timefold.ai](https://docs.timefold.ai).